In [22]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, Input

import os
import numpy as np

In [30]:
np.set_printoptions(precision=3, suppress=True)

In [12]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" #CPU

In [13]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

In [15]:
model = models.Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=1)

1875/1875 [==============================] - 5s 2ms/step - loss: 0.1439 - accuracy: 0.9552


In [31]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

313/313 - 0s - loss: 0.0753 - accuracy: 0.9751
0.9750999808311462


## predict
#### input data : image array (1, 28, 28, 1) float32 
#### output : array (1, 10) float32

In [32]:
test_sample = test_images[0:1]
test_sample.shape, test_sample.dtype, np.min(test_sample), np.max(test_sample)

((1, 28, 28, 1), dtype('float64'), 0.0, 1.0)

In [45]:
predict_result = model.predict(test_sample)
predict_result.shape, predict_result

((1, 10), array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32))

## save as .pb format

In [46]:
saved_model_dir = './mnist'
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./mnist/assets


INFO:tensorflow:Assets written to: ./mnist/assets


## save as .h5 format(simple!)

In [47]:
model.save('./mnist.h5')

In [48]:
!ls *.h5

mnist.h5


## convert .h5 to tflite

In [49]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [50]:
# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [51]:
!ls *.tflite

model.tflite


## load model from tflite

In [52]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp04y24bz1/assets


INFO:tensorflow:Assets written to: /tmp/tmp04y24bz1/assets


In [53]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print('input_details', input_details)
print('output_details', output_details)

input_details [{'name': 'input_2', 'index': 0, 'shape': array([ 1, 28, 28,  1], dtype=int32), 'shape_signature': array([-1, 28, 28,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output_details [{'name': 'Identity', 'index': 20, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


## inference from tflite model

In [54]:
interpreter.set_tensor(input_details[0]['index'], test_sample.astype(np.float32))
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
tflite_results = interpreter.get_tensor(output_details[0]['index'])
tflite_results.shape, tflite_results

((1, 10), array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32))